<a href="https://colab.research.google.com/github/lmoss/onesharp/blob/main/sanity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <span style="color:blue">Sanity: a tool to simplify the construction of 1# programs</span> 
Lawrence S. Moss,
Indiana University


#### This notebook is part of a set of tools for 1#, under development for use in my IU class M584 in this (current) fall 2022 semester.  The main source on 1# is the web text available at https://iulg.sitehost.iu.edu/trm. 

#### The idea of sanity is to make it easier for someone to organize 1# programs and to write the without having to count lines for all of the forward- and backward-transfer statements.

#### The concept and the name come from Jon Bowman, who once took my class and felt that construction 1# programs by hand was crazy, and that counting all the 1's in a long expression "made his eyeballs bleed."

To start, run the next code cells.

In [1]:
#@title
import pandas as pd
from IPython.display import display
import numpy as np

def program_checker(str):
    m = len(str)
    x1 = str[m - 1] == '#'
    x2 = all((str[i] == '1' or str[i] == '#') for i in range(m))
    x3 = (str.find('######') == -1)
    if (x1 and x2 and x3):
        flag = True
    else:
        flag = False
        print('The input ' + str + ' is not a valid 1# program.')
        print('It is not the concatenation of a sequence of instructions in the language.')
        print('So what you are asking for is undefined.')
    return (flag)

def one_or_sharp_check(letter):
    if (letter=="1" or letter=="#"):
        return(True)
    else:
        return(False)

def word_checker(strg):
    answer = all([one_or_sharp_check(x)==True for x in strg])
    return(answer)

def input_checker(input_seq):
    seq = [word_checker(x) for x in input_seq]
    flag = all([word_checker(x) for x in input_seq])
    if not flag:
        print('The input sequence contains words with characters other than 1 and #.')
        print('So what you are asking for is undefined.')
    return(flag)
                                                                                
class Augmented:
    def __init__(self, string, remainders):
        self.string = string
        self.remainders = remainders

class Snapshot:
    def __init__(self, instr_number, regs, proceed,verbose,program_length, step_number):
        self.instr_number = instr_number
        self.regs = regs
        self.proceed = proceed
        self.verbose = verbose
        self.program_length = program_length
        self.step_number = step_number
        
def preparse(xstr):
    b = xstr.string.find('#1')
    xstr.remainders = xstr.remainders + [xstr.string[:(b + 1)]]
    xstr.string = xstr.string[(b + 1):]
    return (xstr)



def parse(y):
    tempx = Augmented(y, [])
    while tempx.string.find('#1') >= 0:
        tempx = preparse(tempx)
    return (tempx.remainders + [tempx.string])

def unparse(p):
    return (''.join(p))

def instruction_type(instruction):
    if instruction[-2:] == '1#':
        return ('add1')
    if instruction[-3:] == '1##':
        return ('add#')
    if instruction[-4:] == '1###':
        return ('forward')
    if instruction[-5:] == '1####':
        return ('backward')
    if instruction[-6:] == '1#####':
        return ('cases')

def tail(list):
    return (list[1:])

def one_step(p, snapshot): # p is parsed
    i = snapshot.instr_number
    r = snapshot.regs
    instruction = p[-1 + i]
    if snapshot.verbose:
        print('Step ' + str(snapshot.step_number) + ':')
        print('Execute instruction ' + str(i) + ':' + " " +
              instruction_gloss(instruction,i-1) 
              + '.')
        if instruction_type(instruction)=='cases':
            billy= len(instruction) - 5
            if snapshot.regs[billy-1] == "":
                print('The register is empty, so we go ahead 1 instruction.')
            elif snapshot.regs[billy-1][0] == "1":
                print('The first symbol in that register is 1,' +
                      ' so we delete that symbol and go forward 2 instructions.')
            elif snapshot.regs[billy-1][0] == "#":
                print('The first symbol in that register is #,' +
                      ' so we delete that symbol and go forward 3 instructions.')     
    t = instruction_type(instruction)
    if t == 'add1':
        snapshot.instr_number = 1 + snapshot.instr_number
        l = len(instruction)
        reg = len(instruction[:(l - 1)])
        snapshot.regs[reg - 1] = snapshot.regs[reg - 1] + '1'
    if t == 'add#':
        snapshot.instr_number = 1 + snapshot.instr_number
        l = len(instruction)
        reg = len(instruction[:(l - 2)])
        snapshot.regs[reg - 1] = snapshot.regs[reg - 1] + '#'
    if t == 'forward':
        l = len(instruction)
        offset = len(instruction[:(l - 3)])
        snapshot.instr_number = offset + snapshot.instr_number
    if t == 'backward':
        l = len(instruction)
        offset = len(instruction[:(l - 4)])
        snapshot.instr_number = (-offset) + snapshot.instr_number
    if t == 'cases':
        l = len(instruction)
        reg = len(instruction[:(l - 5)])
        if snapshot.regs[reg - 1] == '':
            snapshot.instr_number = 1 + snapshot.instr_number
        elif snapshot.regs[reg - 1][0] == '1':
            snapshot.instr_number = 2 + snapshot.instr_number
            snapshot.regs[reg - 1] = tail(snapshot.regs[reg - 1])
        elif snapshot.regs[reg - 1][0] == '#':
            snapshot.instr_number = 3 + snapshot.instr_number
            snapshot.regs[reg - 1] = tail(snapshot.regs[reg - 1])
    if 0< snapshot.instr_number <= len(p):
        snapshot.proceed = True
        if snapshot.verbose == True:
            print_snapshot(snapshot)
    else:
         snapshot.proceed = False
    return (snapshot)


def number_help(instr):
    if instruction_type(instr) == 'add1':
        return (len(instr) - 1)
    if instruction_type(instr) == 'add#':
        return (len(instr) - 2)
    if instruction_type(instr) == 'cases':
        return (len(instr)-5)
    else:
        return (0)


def max_register(p):
    return (max([number_help(instr) for instr in parse(p)]))


def pad(p, register_inputs):
    n = len(register_inputs)
    m = max_register(p)
    extras = ['' for x in range(m - n)]
    bigger = register_inputs + extras
    return (bigger)

def print_snapshot(snap):
    regdf = pd.DataFrame([[snap.regs[n]] for n in range(len(snap.regs))],columns=["contents"])
    regdf.index = np.arange(1, len(regdf) + 1)
    def make_pretty(styler):
        styler.set_properties(**{'background-color': '#FFFFCC'})
        styler.set_properties(**{'text-align': 'left'})
        #styler.set_caption("at the start")
        #styler.hide(axis='index')
        return styler
    display(regdf.style.pipe(make_pretty))  

def step_by_step(word_prog, register_inputs):
    word_prog = word_prog.replace(" ", "")
    register_inputs = [word.replace(" ", "") for word in register_inputs]
    if program_checker(word_prog) and input_checker(register_inputs):
        print('First, here is the program:')
        parse_explain(word_prog)
        print()
        regs = pad(word_prog, register_inputs)
        prog = parse(word_prog)
        N = len(prog)
        snap = Snapshot(1, regs,True,True,N,1)
        print('The computation starts with the register contents shown below.')
        print('The registers include those those which you entered as part of the input')
        print('and also others mentioned in the input program.')
        print_snapshot(snap)
        print()
        while 0 < snap.instr_number < N + 1:
            snap = one_step(prog, snap)
            snap.step_number = (snap.step_number) + 1
        if snap.instr_number <= 0:
            print(
                'The computation has not halted properly ' +
                'because the control went above instruction 1 of the program.'
                 )
        elif (snap.instr_number == (N + 1)) and all(
                snap.regs[i] == ""
                for i in range(1, len(snap.regs))):
            print(
                'The computation then halts properly because' +
                ' the control is just below the last line of the program,')
            print('and because all registers other than R1 are empty.')
            if snap.regs[0] == "":
                print('The output is the empty word.')
            else:
                print('The output is ' + snap.regs[0] + '.')
        else:
            print('This computation does not halt.')
            if snap.instr_number != N + 1:
                print('This is because the program has ' + str(len(prog)) +
                  ' instructions, and control at the end is not one line ' + 
                   'below the bottom of the program.')
                print()
            else:
                not_blank = [
                    i + 1 for i in range(1, len(snap.regs))
                    if snap.regs[i] != ""
                ]
                print('Here is the list of registers whose contents ' +
                      'are not empty at this point, other than R1:' +
                      str(not_blank) + '.')
                print('The register contents at the end are shown above.')


def onesharp(word_prog, register_inputs):
    word_prog = word_prog.replace(" ", "")
    register_inputs = [word.replace(" ", "") for word in register_inputs]  
    if program_checker(word_prog) and input_checker(register_inputs):
        register_inputs = [word.replace(" ", "") for word in register_inputs]
        regs = pad(word_prog, register_inputs)
        prog = parse(word_prog)
        N = len(prog)
        snap = Snapshot(1, regs,True,False, N, 1)
        while snap.proceed:
            snap = one_step(prog, snap)
            snap.step_number = (snap.step_number)+1
        if (snap.instr_number == (N + 1)) and all(
                snap.regs[i] == "" for i in range(1, len(snap.regs))):
            return ((snap.regs)[0])
        else:            
            print("This is undefined.")
            print("The register contents at the end are shown below.")
            print_snapshot(snap)
    else:
        return('undefined')


def instruction_gloss(instr,line):
    if instruction_type(instr) == 'add1':
        return ('add 1 to R' + str(len(instr) - 1))
    if instruction_type(instr) == 'add#':
        return ('add # to R' + str(len(instr) - 2))
    if instruction_type(instr) == 'forward':
        w = len(instr) - 3
        return ('go forward ' + str(w) + ' to instruction ' + str(w+line+1))
    if instruction_type(instr) == 'backward':
        w = len(instr) - 4
        return ('go backward ' + str(w) + ' to instruction ' + str(line - w+1))
    if instruction_type(instr) == 'cases':
        return ('cases on R' + str(len(instr) - 5))

def expanded(gorp):
    pgorp = parse(gorp)
    wwgorp = [[pgorp[x],instruction_gloss(pgorp[x],x)] for x in range(len(pgorp))]
    return(wwgorp)

def parse_explain(prog):
    df = pd.DataFrame(expanded(prog),
                      columns=["instruction", 'explanation'])
    df.index = np.arange(1, len(df) + 1)
    def make_pretty(styler):
                styler.set_properties(**{'background-color': '#C9DFEC'})        
                styler.set_properties(**{'text-align': 'left'})
                return styler
    display(df.style.pipe(make_pretty))
    #display(df)
    
def ones(n):
  return(unparse(['1' for i in range(n)])) 
clear_2 = '11#####111###11####111####'
clear_3 = '111#####111###11####111####'
clear_4 = '1111#####111###11####111####'
move_1_2= '1#####111111###111###11##1111####11#111111####'
move_1_3= '1#####111111###111###111##1111####111#111111####'
move_1_4= '1#####111111###111###1111##1111####1111#111111####'
move_2_1= '11#####111111###111###1##1111####1#111111####'
move_2_3= '11#####111111###111###111##1111####111#111111####'
move_2_4= '11#####111111###111###1111##1111####1111#111111####'
move_3_1= '111#####111111###111###1##1111####1#111111####'
move_3_2= '111#####111111###111###11##1111####11#111111####'
move_3_4= '111#####111111###111###1111##1111####1111#111111####'
move_4_1= '1111#####111111###111###1##1111####1#111111####'
move_4_2= '1111#####111111###111###11##1111####11#111111####'
move_4_3= '1111#####111111###111###111##1111####111#111111####'
copy_1_2_3 = '1#####11111111###1111###11##111##11111####11#111#11111111####111#####111111###111###1##1111####1#111111####'
copy_1_2_4 ='1#####11111111###1111###11##1111##11111####11#1111#11111111####1111#####111111###111###1##1111####1#111111####'
copy_1_3_4='1#####11111111###1111###111##1111##11111####111#1111#11111111####1111#####111111###111###1##1111####1#111111####'
copy_2_3_4 = '11#####11111111###1111###111##1111##11111####111#1111#11111111####1111#####111111###111###11##1111####11#111111####'



In [18]:
#@title


# This code cell contains macros for several common 1# programs; the macros
#   allow a programmer to choose the registers that a program will operate on


# clear
# =====
# reg     : positive integer, the index of the register to clear
# returns : string, the 1# program to clear register ``reg''
def clear(reg):
  p = ones(reg)+'#####' # Cases on register ``reg''
  p += '111###'         # Go forward three
  p += '11####'         # Go backward two
  p += '111####'        # Go backward three
  return p

# move
# ====
# src     : positive integer, the index of the source register
# dst     : positive integer, the index of the destination register
# returns : string, the 1# program to move the contents of register ``src'' to
#         :   register ``dst''
def move(src, dst):
  p = ones(src)+'#####' # Cases on register ``src''
  p += '11111 1###'     # Go forward six
  p += '111###'         # Go forward three
  p += ones(dst)+'##'   # Append ``#'' to register ``dst''
  p += '1111####'       # Go backward four
  p += ones(dst)+'#'    # Append ``1'' to register ``dst''
  p += '11111 1####'    # Go backward six
  return p

# copy
# ====
# src     : positive integer, the index of the source register
# dst     : positive integer, the index of the destination register
# tmp     : positive integer, the index of the temporary register
# returns : string, the 1# program to copy the contents from register ``src'' to
#         :  register ``dst'' using register ``tmp'' as temporary scratch space
def copy(src, dst, tmp):
  p = ones(src)+'#####' # Cases on register ``src''
  p += '11111 111###'   # Go forward eight 
  p += '1111###'        # Go forward four
  p += ones(dst)+'##'   # Append ``#'' to register ``dst''
  p += ones(tmp)+'##'   # Append ``#'' to register ``tmp''
  p += '11111####'      # Go backward five
  p += ones(dst)+'#'    # Append ``1'' to register ``dst''
  p += ones(tmp)+'#'    # Append ``1'' to register ``tmp''
  p += '11111 111####'  # Go backward eight
  p += move(tmp, src)   # Move the contents of register ``tmp'' to register
                        #   ``src''
  return p

# succ
# ====
# reg     : positive integer, the index of the register for whose contents
#         :   the successor will be calculated
# tmp     : positive integer, the index of the temporary register
# returns : string, the 1# program which will find the successor of the
#         :   (nonnegative) backward binary integer in register ``reg''
#         :   and place the successor back into register ``reg'' while using
#         :   register ``tmp'' as temporary scratch space
def succ(reg, tmp):
  p = ones(reg)+'#####'      # Cases on register ``reg''
  p += '11111 11111 111###'  # Go forward thirteen
  p += '11111 11111###'      # Go forward ten 
  p += ones(tmp)+'#'         # Append ``1'' to register ``tmp''
  p += move(reg, tmp)        # Move the remaining contents of register ``reg''
                             #   to register ``tmp''; seven instructions
  p += '1111###'             # Go forward four
  p += ones(tmp)+'##'        # Append ``#'' to register ``tmp''
  p += '11111 11111 111####' # Go backward thirteen
  p += ones(tmp)+'#'         # Append ``1'' to register ``tmp''
  p += move(tmp, reg)        # Move the contents of register ``tmp'' to
                             #   register ``reg''
  return p

# cmp
# ===
# dst     : positive integer, the index of the destination register which
#         :   also contains the first string in the comparison
# src     : positive integer, the index of the source register which contains
#         :   the second string in the comparison
# returns : string, the 1# program which will compare the strings in registers
#         :   ``dst'' and ``src,'' decide if they are symbol-for-symbol the
#         :   same, and if they are the same write a ``1'' back into register
#         :  ``dst'' but otherwise leave register ``dst'' empty
def cmp(dst, src):
  p = ones(dst)+'#####'       # Cases on register ``dst''
  p += '11111 1###'           # Go forward six
  p += '11111 1111###'        # Go forward nine
  p += ones(src)+'#####'      # Cases on register ``src''
  p += '11111 11111 1###'     # Go forward eleven
  p += '11111 11111###'       # Go forward ten
  p += '11111 1####'          # Go backward six
  p += ones(src)+'#####'      # Cases on register ``src''
  p += '11111 11111 11111###' # Go forward fifteen
  p += '11111 1###'           # Go forward six
  p += '11111###'             # Go forward five
  p += ones(src)+'#####'      # Cases on register ``src''
  p += '111###'               # Go forward three
  p += '11111 11111 111####'  # Go backward thirteen
  p += '1###'                 # Go forward one; no op
  p += ones(dst)+'#####'      # Cases on register ``dst''
  p += '111###'               # Go forward three
  p += '11####'               # Go backward two
  p += '111####'              # Go backward three
  p += ones(src)+'#####'      # Cases on register ``src''
  p += '1111###'              # Go forward four
  p += '11####'               # Go backward two
  p += '111####'              # Go backward three
  p += ones(dst)+'#'          # Append ``1'' to register ``dst''
  return p

# add
# ===
# dst     : positive integer, the index of the destination register which also
#         :   must contain the first addend
# src     : positive integer, the index of the source register which must
#         :   contain the second addend
# tmp     : positive integer, the index of the temporary register
# returns : string, the 1# program which will calculate the sum, in backward
#         :   binary, of the contents of registers ``dst'' and ``src'' and store
#         :   the result back into register ``dst'' while using register ``tmp''
#         :   as temporary scratch space
def add(dst, src, tmp):
  p = ones(dst)+'#####'  # Cases on register ``dst''
  p += '111###'          # Go forward three
  p += '11111 1###'      # Go forward six
  p += '11111 1111###'   # Go forward nine
  p += ones(src)+'#####' # Cases on register ``src''
                         # Go forward thirty-five
  p += '11111 11111 11111 11111 11111 11111 11111###'
                         # Go forward twenty-six
  p += '11111 11111 11111 11111 11111 1###'
                         # Go forward twenty-seven
  p += '11111 11111 11111 11111 11111 11###'
  p += ones(src)+'#####' # Cases on register ``src''
                         # Go forward twenty-three
  p += '11111 11111 11111 11111 111###'
                         # Go forward twenty-eight
  p += '11111 11111 11111 11111 11111 111###'
                         # Go forward twenty-one
  p += '11111 11111 11111 11111 1###'
  p += ones(src)+'#####' # Cases on register ``src''
                         # Go forward twenty-one
  p += '11111 11111 11111 11111 1###'
                         # Go forward eighteen
  p += '11111 11111 11111 111###'
                         # Go forward nineteen
  p += '11111 11111 11111 1111###'
  p += ones(dst)+'#####' # Cases on register ``dst''
  p += '111###'          # Go forward three
  p += '11111 1###'      # Go forward six
  p += '11111 1111###'   # Go forward nine
  p += ones(src)+'#####' # Cases on register ``src''
                         # Go forward eleven
  p += '11111 11111 1###'
                         # Go forward sixteen
  p += '11111 11111 11111 1###'
  p += '11111 1111###'   # Go forward nine
  p += ones(src)+'#####' # Cases on register ``src''
                         # Go forward thirteen
  p += '11111 11111 111###'
  p += '11111 11111###'  # Go forward ten
                         # Go forward eleven
  p += '11111 11111 1###'
  p += ones(src)+'#####' # Cases on register ``src''
  p += '111###'          # Go forward three
  p += '11111 111###'    # Go forward eight
  p += '1###'            # Go forward one; no op
  p += ones(tmp)+'#'     # Append ``1'' to register ``tmp''
                         # Go backward thirty-three
  p += '11111 11111 11111 11111 11111 11111 111####'
  p += ones(tmp)+'##'    # Append ``#'' to register ``tmp''
                         # Go backward thirty-five
  p += '11111 11111 11111 11111 11111 11111 11111####'
  p += ones(tmp)+'#'     # Append ``1'' to register ``tmp''
                         # Go backward twenty-one
  p += '11111 11111 11111 11111 1####'
  p += ones(tmp)+'##'    # Append ``#'' to register ``tmp''
                         # Go backward twenty-three
  p += '11111 11111 11111 11111 111####'
  p += move(tmp, dst)    # Move the contents of register ``tmp'' to register
                         #   ``dst''
  return p

In [51]:
#@title
def findme(pair,pair_list): 
  x = [i for i in range(len(pair_list)) if pair_list[i][0]==pair[1]]
  return x[0]

def add_placeholder(line):
  keywords = ['cases','add1','add#','goto','non_label']
  if line[0] in keywords:
    return(['non_label']+line)
  elif line[0][0] in ['1','#']:
    return(['non_label']+line)  
  else:
    return(line)

def flatten(tuple):
  b = len(tuple)
  if b > 1 and tuple[1] == "cases":
    n = tuple[2]
    x = ones(n)+'#####'
    return [[tuple[0],x], [tuple[0]+'@1',tuple[3]], [tuple[0]+'@2',tuple[4]], [tuple[0]+'@3',tuple[5]]]
  elif b > 1 and tuple[1]=='add1':
    return([[tuple[0],ones(tuple[2])+'#']])
  elif b > 1 and tuple[1]=='add#':
    return([[tuple[0],ones(tuple[2])+'##']])
  elif b > 1 and tuple[1]=='goto':
    return [[tuple[0],tuple[2]]]
  elif tuple[1][0] in ['1', '#']:
    k = parse(tuple[1])
    m = len(k)
    return([[tuple[0],k[j]] for j in range(m)])


def ones(n):
  return(unparse(['1' for i in range(n)])) 

def resolve(index,pair_list):
  pair_in_list = pair_list[index]
  #print("pair_in_list " + str(pair_in_list))
  first_char = pair_in_list[1][0]
  #print(first_char)
  if first_char == '1' or first_char == '#':
    return pair_in_list[1]
  elif pair_in_list[1]=='end':
    n = len(pair_list) - index
    return(ones(n)+'###')
  else:
    k = findme(pair_in_list,pair_list)
    if k > index:
      return(ones(k-index)+'###')
    if k < index:
      return(ones(index-k)+'####')


def sanity(line_list):
  w = [add_placeholder(line) for line in line_list]
  #print("[add_placeholder(line) for line in line_list] is")
  #print(w)
  s1 = [flatten(line) for line in w]
  #print('[flatten(line) for line in w] is')
  #print(s1)
  t1 = [item for sublist in s1 for item in sublist]
  #print('[item for sublist in s1 for item in sublist] is')
  #print(t1)
  n = len(t1)
  #print('len of t1 = ' + str(n))
  u1 = [resolve(i,t1) for i in range(n)] 
  return(unparse(u1))      

As a way to show what the tool does, we'll go through an example.  Let's write a program that takes a word 

$$ w = w_1 w_2 \cdots w_n $$

in R1 and reverses it.  Our program will work as follows.   It processes the letters in $w$ in order, using a loop. At the end of the $i$th iteration, we'll have $w_{i+1}\cdots w)n$ in R1, and its prefix will be in R2 *backwards*: $w_{i}\cdots w_2 w_1$.  

The $i$th step itself copies $w_i$ into R3, and then (in order to put that symbol on the *front* of $w_{i}\cdots w_2 w_1$, moves R2 on the end of R3, and then R3 back to R2.
Once we have gone through the original $w$ in this fashion, R1 will be empty, and R2 will contain its reversal.  So we close by moving R2 back to the now-empty R1.

With this in mind, have a look at the following array 'reverse_idea', itself containing 8 arrays.

In [ ]:
reverse_idea = [
    ['top', 'cases', 1, 'move_back', 'found_a_one', 'found_a_sharp'],
    ['found_a_one','111#'],
    ['goto', 'move_phase'],
    ['found_a_sharp',  '111##'],
    ['goto', 'move_phase'],
    ['move_phase', move_2_3 + move_3_2],
    ['goto', 'top'],
    ['move_back', move_2_1]
]


We have 8 *lines*. But a line is not the same as in instruction: lines 6 and 8 each contain move programs that are bigger than a single instruction.  Lines 2, 4 5, and 6 each begin with a *label*.  Labels are strings that other parts of the program could point to.  For example, the first line is a case statement 1#####, and it also contains the information that if R1 is empty, we should go to whichever line has the label 'move_phase'. (That would be the line named 'move_stuff_around'.) The first line also tells us that if R1 begins with 1 we should (delete is and) go to the line containing 'first-is_one'.   Note also that 'goto' is not a label.  

Here is how these lines are used:



In [ ]:
rev = sanity(reverse_idea)
# This run 'sanity' on 'reverse_idea', calling the result 'rev'.
# We can refer to it in the rest of this notebook by 'rev'.
# For example we can display our new program
rev

'1#####1111111111111111111111###11###111###111#111###111##1###11#####111111###111###111##1111####111#111111####111#####111111###111###11##1111####11#111111####1111111111111111111111####11#####111111###111###1##1111####1#111111####'

Now the program which we just constructed can be run, as usual:

In [ ]:
onesharp(rev,['1####'])

'####1'

In [ ]:
# Here is a way to write the program 'clear_1':
sanity([
    ['top', 'cases',1,'empty', 'one','hash'],
    ['empty', 'goto', 'end'],
    ['one','goto', 'top'],
    ['hash', 'goto', 'top'],
])

'1#####111###111###111###111###11111####111111####'

Notice that in the last examples we had a line

   ['empty', 'goto', 'end']

In this, 'end' is not a label in any of the four lines.
Indeed, 'end' is a special string in this program.   We can use 'end' in connection with 'goto', and also in one of the branches of a 'cases' statement.

Other things to know: instead of (for example) 11#, we can write it in words as in the third line below.   Finally, all numbers in this program must be entered without quotes.

In [ ]:
d = [
    ['top','cases',1,'empty', 'one','hash'],
    ['empty', 'goto', 'moveback'],
    ['one', 'add1', 2],
    ['111#111##'],
    ['goto', 'top'],
    ['hash', 'add#', 2],
    ['111#111##111##'],
    ['goto', 'top'],
    ['moveback', move_3_1+move_2_1] 
]
dg = sanity(d)

In [35]:
onesharp(dg,['11#'])

'1#1#1##11#'

###Summary: here are some examples of 'lines' that the tool can handle:

    ['top','cases',1,'empty', 'one_found','hash_found'],
    ['empty', 'goto', 'moveback'],
    ['one_found', 'add1', 2],
    ['111#111##'],
    ['goto', 'top'],
    ['hash_found', 'add#', 2],
    ['111#111##111##'],
    ['goto', 'end'],
    ['moveback', move_3_1+move_2_1]
  
A line can be snippet of 1# code surrounded by quotes.  It can also be a Python expression like 

   move_3_1 + move_2_1
   
that denotes a 1# word.

A line may begin with a *label* like 'top', or 'moveback' 
A label should not begin with '1' or '#, and it should not be one of the strings 'goto', 'end', 'add1', or 'add#'.

*Labels are optional*, except a "cases" instruction
must have a number and then three labels.  

A line also can have the word 'goto' followed by a label or 'end'.

A line can have 'add1' or 'add#' followed by a number (without quotes).

Every label used inside a 'cases' or 'goto' statement must be the first label in some line.  Otherwise, the program will raise an error.


### Two more example Sane programs

In [ ]:
# This code cell contains a Sane program which multiplies the contents of
#   registers one and two and stores the product back into register one

sane_multiply = [
  [move(1,4)],
  ['1##'],
  [copy(2,5,10)],
  ['111##'],
  [copy(3,6,10)],
  [cmp(2,3)],
  ['multiply_loop', 'cases', 2, 'empty', 'one', 'sharp'],
    ['empty', copy(4,7,10)],
      [add(1,4,10)],
      [move(7,4)],
      [copy(5,2,10)],
      [succ(6,10)],
      [copy(6,3,10)],
      [cmp(2,3)],
      ['goto', 'multiply_loop'],
    ['one', 'goto', 'epilogue'],
    ['sharp', 'goto', 'end'], # We shouldn't reach here because cmp shold never
                              #   write sharp into register two
  ['epilogue', clear(4)],
    [clear(5)],
    [clear(6)]
]
onesharp_multiply = sanity(sane_multiply)
onesharp(onesharp_multiply, ['11', '1#1']) # 11*1#1 = 1111 <==> 3*5 = 15

In [30]:
# This code cell contains a Sane program which exponentiates the contents of
#   register one to the power of the contents of register two and stores the
#   result back into register one

sane_exponentiate = [
  [move(1,14)],
  [ones(11)+'#'],
  [move(2,12)],
  [copy(12,15,20)],
  [ones(13)+'##'],
  [copy(13,16,20)],
  [cmp(12,13)],
  ['exponentiate_loop', 'cases', 12, 'empty', 'one', 'sharp'],
    ['empty', move(11,1)],
      [copy(14,2,20)],
      [onesharp_multiply],
      [move(1,11)],
      [copy(15,12,20)],
      [succ(16,20)],
      [copy(16,13,20)],
      [cmp(12,13)],
      ['goto', 'exponentiate_loop'],
    ['one', 'goto', 'epilogue'],
    ['sharp', 'goto', 'end'], # We shouldn't reach here because cmp shold never
                              #   write sharp into register two
  ['epilogue', move(11,1)],
    [clear(14)],
    [clear(15)],
    [clear(16)]
]
onesharp_exponentiate = sanity(sane_exponentiate)
onesharp(onesharp_exponentiate, ['11', '1#1']) # 11^1#1 = 11##1111 <==> 3^5 = 243

[flatten(line) for line in w] is
[[['non_label', '1#####'], ['non_label', '11111 1###'], ['non_label', '111###'], ['non_label', '11111111111111##'], ['non_label', '1111####'], ['non_label', '11111111111111#'], ['non_label', '11111 1####']], [['non_label', '11111111111#']], [['non_label', '11#####'], ['non_label', '11111 1###'], ['non_label', '111###'], ['non_label', '111111111111##'], ['non_label', '1111####'], ['non_label', '111111111111#'], ['non_label', '11111 1####']], [['non_label', '111111111111#####'], ['non_label', '11111 111###'], ['non_label', '1111###'], ['non_label', '111111111111111##'], ['non_label', '11111111111111111111##'], ['non_label', '11111####'], ['non_label', '111111111111111#'], ['non_label', '11111111111111111111#'], ['non_label', '11111 111####'], ['non_label', '11111111111111111111#####'], ['non_label', '11111 1###'], ['non_label', '111###'], ['non_label', '111111111111##'], ['non_label', '1111####'], ['non_label', '111111111111#'], ['non_label', '11111 1####

'11##1111'

In [34]:
onesharp(onesharp_exponentiate, ['11', '###1']) # 11^###1 = 1####1#11##11 <==> 3^8 = 6561
# 6561 base 2 is 1100110100001

'1####1#11##11'

In [78]:
 pre_pred = [
 ['top', 'cases', 1, 'first_end', 'first_one', 'first_hash'],
 ['first_one', 'cases', 1, 'hash_is_it', 'returnA','returnB'],
 ['hash_is_it', '1##'],
 ['goto', 'second_end'],
 ['returnA', '11#11#'],
 [move_1_2 + move_2_1],
 ['goto', 'end'], 
 ['returnB', '11#11##'],
 [move_1_2 + move_2_1],
 ['goto', 'end'], 
 ['first_hash', 'cases', 1, 'first_end', 'hash_one', 'hash_hash'],
 ['hash_one','11##'],
 ['hash_hash','1###'],
 ['second_end', '1111#'],
 ['goto', 'end'],
 ['first_end', '111#'],
 ]

In [83]:
onesharp(sanity(pre_pred), ['#1'])

This is undefined.
The register contents at the end are shown below.


,contents
1,
2,
3,1
4,


In [141]:
pred = [
     ['top','cases', 1, 'a', 'b','c'],   
     ['a', 'goto', 'end'],
     ['b', 'cases', 1, 'oe', 'oo', 'oh'],
     ['oe', '1##'],
     ['goto', 'end'],
     ['oo', '11#11#'+move_1_2+move_2_1],
     ['goto', 'main'],
     ['oh', '11#11##'+move_1_2+move_2_1],
     ['goto', 'main'],
     ['c', 'cases', 1, 'he', 'ho', 'hh'],
     ['he', '1##'],
     ['goto', 'end'],
     ['ho', '11##11#'+move_1_2+move_2_1],
     ['goto', 'main'],
     ['hh', '11##11##'+move_1_2+move_2_1],
     ['goto', 'main'],    
     ['main', 'cases', 1, 'empty', 'one','hash'],
     ['empty', move_2_1],
     ['goto', 'end'],
     ['one', '11##'],
     [move_1_2 + move_2_1],
     ['goto', 'end'],
     ['hash', '11#'],
     ['borrowing', 'cases', 1, 'borrowing_empty', 'borrowing_one', 'borrowing_hash'],
     ['borrowing_empty', move_2_1],
     ['goto', 'end'],
     ['borrowing_one', '11##'],
     [move_1_2 + move_2_1],
     ['goto', 'end'],
     ['borrowing_hash', '11#'],
     ['goto','borrowing']
]

In [142]:
p1 = sanity(pred)


In [150]:
onesharp(p1,['1#'])

'##'

In [156]:
id = [ ['all_h', 'cases', 1, 'a', 'b', 'c'],
       ['a', '1##'+clear_2],
       ['goto', 'end'],
       ['b', '11#'],
       [move_1_2+move_2_1],
       ['goto', 'end'],
       ['c', '11##'],
       ['goto', 'all_h']
      ]

rectify=sanity(id)

In [160]:
onesharp(rectify,['##1'])

'##1'

In [161]:
pr = p1 + rectify

In [168]:
onesharp(pr,['1##'])

'#'

In [169]:
pr

'1#####111###111###111111111111111111111111111111111111111111###11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111###1#####111###1111###11111111111111111111###1##11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111###11#11#1#####111111###111###11##1111####11#111111####11#####111111###111###1##1111####1#111111####1111111111111111111111111111111111111111111111111111111111###11#11##1#####111111###111###11##1111####11#111111####11#####111111###111###1##1111####1#111111####11111111111111111111111111111111111111111###1#####111###1111###11111111111111111111###1##1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111###11##11#1#####111111###111###11##1111####11#111111####11#####111111###111###1##1111####1#111111####111111111111111111###11##11##1#####111111###111###11##1111####11#111111